In [1]:
import numpy as np
import pandas as pd
from math import floor
from scipy.optimize import minimize_scalar
from itertools import combinations

In [2]:
import numpy as np
from scipy.linalg import inv, det, pinv
from scipy.stats import multivariate_normal

def ind_normal_sigma_fixed_marginal_fun_indi(zSigmaz_S, tau, p_S, det_S):
    result = p_S / 2.0 * np.log(tau) - 0.5 * np.log(det_S) + zSigmaz_S / 2.0
    return result

def ind_Normal_fixed_sigma_marginal_external(index_vec_input, Sigma, z, tau, p_S, y_sigma):
    index_vec = index_vec_input - 1
    Sigma_S = Sigma[np.ix_(index_vec, index_vec)]
    A = tau * np.eye(p_S)
    
    det_S = det(Sigma_S + A)
    Sigma_S_inv = inv(Sigma_S + A)
    
    sub_z = z[index_vec]
    zSigmaz_S = np.dot(sub_z.T, np.dot(Sigma_S_inv, sub_z))
    
    b = ind_normal_sigma_fixed_marginal_fun_indi(zSigmaz_S, tau, p_S, det_S)
    
    results = b
    
    return results


def outlier_ind_Normal_marginal_external(index_vec_input, Sigma, z, tau, p_S, y_sigma):
    index_vec = index_vec_input - 1
    
    Sigma_S = Sigma[np.ix_(index_vec, index_vec)]
    A = tau * np.eye(p_S)
    
    Sigma_S_I_inv = pinv(Sigma_S + A, rtol=0.00001)
    Sigma_S_inv = pinv(Sigma_S, rtol=0.00001)
    
    det_S = np.abs(det(Sigma_S_inv))
    det_I_S = np.abs(det(Sigma_S_I_inv))
    
    sub_z = z[index_vec]
    zSigmaz_S = np.dot(sub_z, np.dot(Sigma_S_inv, sub_z))
    zSigmaz_I_S = np.dot(sub_z, np.dot(Sigma_S_I_inv, sub_z))
    
    b = 0.5 * (np.log(det_S) + np.log(det_I_S)) - 0.5 * (zSigmaz_S - zSigmaz_I_S)
    results = b
    
    return results


In [3]:
def add_function(S_sub, y):
    results = [np.sort(np.concatenate(([x],y))) for x in S_sub]
    return np.array(results)

def set_gamma_func_base(S, p):
    set_gamma = {}

    # set of gamma-
    if len(S) == 0:
        set_gamma[0] = None
        set_gamma[1] = np.arange(0, p ).reshape(-1, 1)
        set_gamma[2] = None

    if len(S) == 1:
        S_sub = np.setdiff1d(np.arange(0, p ), S)
        set_gamma[0] = None
        set_gamma[1] = add_function(S_sub, S)
        set_gamma[2] = S_sub.reshape(-1, 1)

    if len(S) > 1:
        S_sub = np.setdiff1d(np.arange(0, p ), S)
        S = np.sort(S)
        set_gamma[0] = np.array(list(combinations(S, len(S) - 1)))
        set_gamma[1] = add_function(S_sub, S)
        xs = np.vstack([add_function(S_sub, row) for row in set_gamma[1]])
        set_gamma[2] = xs

    return set_gamma

def set_gamma_func_conditional(input_S, condition_index, p):
    set_gamma = {}
    S = np.setdiff1d(input_S, condition_index)

    # set of gamma-
    if len(S) == 0:
        S_sub = np.setdiff1d(np.arange(0, p), condition_index)
        set_gamma[0] = None
        set_gamma[1] = S_sub.reshape(-1, 1)
        set_gamma[2] = None

    if len(S) == 1:
        S_sub = np.setdiff1d(np.arange(0, p), input_S)
        set_gamma[0] = None
        set_gamma[1] = add_function(S_sub, S)
        set_gamma[2] = S_sub.reshape(-1, 1)

    if len(S) > 1:
        S_sub = np.setdiff1d(np.arange(0, p), input_S)
        S = np.sort(S)
        set_gamma[0] = np.array(list(combinations(S, len(S) - 1)))
        set_gamma[1] = add_function(S_sub, S)
        xs = np.vstack([add_function(S_sub, row) for row in set_gamma[0]])
        set_gamma[2] = xs

    return set_gamma

def set_gamma_func(input_S, p, condition_index=None):
    if condition_index is None:
        results = set_gamma_func_base(input_S, p)
    else:
        results = set_gamma_func_conditional(input_S, condition_index, p)
    return results

def index_fun_internal(x):
    y=np.sort(x)
    y=y.astype(str)
    l=','.join(y)
    return l

def index_fun(y):
    l=np.array([index_fun_internal(x) for x in y])
    return l

In [4]:
def ridge_fun(x, Sigma, modi_ld_S, test_S, z, outlier_tau):
    temp_Sigma = Sigma.copy()
    temp_ld_S = x * modi_ld_S + (1 - x) * np.eye(len(modi_ld_S))
    temp_Sigma[np.ix_(test_S, test_S)] = temp_ld_S
    l = outlier_likelihood(index_vec_input=test_S + 1, Sigma=temp_Sigma, z=z, tau=outlier_tau, p_S=len(test_S), y_sigma=1)
    return -l

def prior_dist(t, lambda_val, p):
    l = t.split(",")
    dim_model = len(l)
    if t=="": dim_model=0
    result = dim_model * np.log(lambda_val) + np.math.lgamma(p - dim_model+1) - np.math.lgamma(p+1)
    return result

In [5]:
ld=pd.read_csv("../03_unit_R/01_test_ld.csv",header=None)
ld=np.array(ld)
z=pd.read_csv("../03_unit_R/01_test_z.csv")
z=np.array(z.iloc[:,1])

In [6]:
Max_Model_Dim=2e+5
all_inner_iter=10
epsilon_threshold=1e-5
num_causal=10
y_var=1
tau=0.04
outlier_switch=True
outlier_BF_index=1/3.2


z=z
ld_matrix=ld
epsilon=0.00021
MaxModelDim=2.5e5
lambda_ = 1
outlier_switch=outlier_switch
tau=tau
num_causal = num_causal
y_var=y_var
inner_all_iter = all_inner_iter

input_conditional_S_list = None

In [7]:
p=len(z)
marginal_likelihood = ind_Normal_fixed_sigma_marginal_external  # Assuming this function is defined elsewhere
tau_sample = tau
if (outlier_switch == True):  # Set this to False if not using outlier likelihood
    outlier_likelihood = outlier_ind_Normal_marginal_external  # Assuming this function is defined elsewhere
    outlier_tau = tau_sample

B = Max_Model_Dim
stored_bf = 0
Sigma =ld_matrix

S = []

null_model = ""
null_margin = prior_dist(null_model, lambda_val=lambda_, p=p)

B_list = pd.DataFrame(
    {'set_gamma_margin': [null_margin],
    'matrix_gamma': [""]})

if input_conditional_S_list==None:
    conditional_S=[]
else:
    conditional_S = input_conditional_S_list 
    S = conditional_S


#MAIN LOOP

In [8]:
h_ind=0
ind_inner_iter=0
for ind_inner_iter in range(0,inner_all_iter):
    for h_ind in range(0,10):
        set_gamma=set_gamma_func(input_S=S, p=p, condition_index=conditional_S) 

        if conditional_S is None:
            working_S=S
        else:
            working_S = np.sort(np.setdiff1d(S, conditional_S)).astype(int)

        set_gamma_margin = {}
        set_gamma_prior = {}
        matrix_gamma = {}

        if len(working_S)!=0:
            S_model = ','.join(np.sort(working_S).astype(int).astype(str))
            p_S = len(working_S)
            current_log_margin = marginal_likelihood(np.array(working_S)+1, Sigma, z, tau=tau_sample, p_S=p_S, y_sigma=y_var)+prior_dist(S_model, lambda_val=lambda_, p=p)
        else:
            current_log_margin = prior_dist(null_model, lambda_val=lambda_, p=p)

        set_gamma_margin = [None,None,None]
        set_gamma_prior = [None,None,None]
        matrix_gamma = [None,None,None]

        for i in range(0, len(set_gamma)):
            if (set_gamma[i] is not None):
                matrix_gamma[i]=index_fun(set_gamma[i])
                p_S = set_gamma[i].shape[1]
                set_gamma_margin[i]=np.apply_along_axis(marginal_likelihood, 1, set_gamma[i]+1, Sigma=Sigma, z=z, tau=tau_sample, p_S=p_S,
                                        y_sigma=y_var)
                set_gamma_prior[i]=np.array([prior_dist(model, lambda_val=lambda_, p=p) for model in matrix_gamma[i]])
                set_gamma_margin[i] = set_gamma_prior[i] + set_gamma_margin[i]
            else:
                set_gamma_margin[i]=np.array(null_margin)
                set_gamma_prior[i]=0
                matrix_gamma[i]=np.array(null_model)

        columns = ['set_gamma_margin', 'matrix_gamma']
        add_B = pd.DataFrame(columns=columns)

        for i in range(len(set_gamma)):
            if type(set_gamma_margin[i].tolist())==list:
                new_row = pd.DataFrame({'set_gamma_margin': set_gamma_margin[i].tolist(),
                                'matrix_gamma': matrix_gamma[i].tolist()})
                add_B = pd.concat([add_B, new_row], ignore_index=True)
            else:
                new_row = pd.DataFrame({'set_gamma_margin': [set_gamma_margin[i].tolist()],
                                'matrix_gamma': [matrix_gamma[i].tolist()]})
                add_B = pd.concat([add_B, new_row], ignore_index=True)


        # Add visited models into the storage space of models
        B_list = pd.concat([B_list, add_B], ignore_index=True)
        B_list=B_list.drop_duplicates(subset='matrix_gamma',ignore_index=True)
        B_list=B_list.sort_values(by='set_gamma_margin',ignore_index=True, ascending=False)

        if len(working_S)==0:
            # Create a DataFrame set.star
            set_star = pd.DataFrame({
                'set_index': [0, 1, 2],
                'gamma_set_index': [np.nan, np.nan, np.nan],
                'margin': [np.nan, np.nan, np.nan]
            })

            # Assuming set.gamma.margin and current.log.margin are defined
            aa = set_gamma_margin[1]
            aa = aa - aa[np.argmax(aa)]

            min_half_len = min(len(aa), floor(p/2))
            decr_ind=np.argsort(np.exp(aa))[::-1]
            decr_half_ind=decr_ind[:min_half_len]

            probs=np.exp(aa)[decr_half_ind]

            chosen_index=np.random.choice(decr_half_ind, 1, p=probs/np.sum(probs))
            set_star.at[1, 'gamma_set_index'] = chosen_index[0]
            set_star.at[1, 'margin'] = set_gamma_margin[1][chosen_index[0]]

            S = set_gamma[1][chosen_index[0]].tolist()

        else:
            set_star = pd.DataFrame({
            'set_index': [0, 1, 2],
            'gamma_set_index': [np.nan, np.nan, np.nan],
            'margin': [np.nan, np.nan, np.nan]
            })
            for i in range(0,3):
                aa = set_gamma_margin[i]
                if np.size(aa)>1:
                    aa = aa - aa[np.argmax(aa)]
                    chosen_index = np.random.choice(range(0, np.size(set_gamma_margin[i])), 1, p=np.exp(aa)/np.sum(np.exp(aa)))
                    set_star.at[i, 'gamma_set_index'] = chosen_index
                    set_star.at[i, 'margin'] = set_gamma_margin[i][chosen_index]
                else:
                    chosen_index=0
                    set_star.at[i, 'gamma_set_index'] = chosen_index
                    set_star.at[i, 'margin'] = set_gamma_margin[i]

            if outlier_switch:
                for i in range(1, len(set_gamma)):
                    test_log_BF=100
                    while True:
                        aa = set_gamma_margin[i]
                        aa = aa - aa[np.argmax(aa)]
                        chosen_index = np.random.choice(range(0, np.size(set_gamma_margin[i])), 1, p=np.exp(aa)/np.sum(np.exp(aa)))
                        set_star.at[i, 'gamma_set_index'] = chosen_index
                        set_star.at[i, 'margin'] = set_gamma_margin[i][chosen_index]

                        test_S = set_gamma[i][int(chosen_index),:]

                        modi_Sigma = Sigma.copy()
                        temp_Sigma = Sigma.copy()
                        if np.size(test_S) > 1:
                            modi_ld_S = modi_Sigma[test_S][:,test_S]

                            result =  minimize_scalar(ridge_fun, bounds=(0,1), args=(Sigma, modi_ld_S, test_S, z, outlier_tau), method='bounded')
                            modi_ld_S = result.x * modi_ld_S + (1 - result.x) * np.eye(len(modi_ld_S))

                            modi_Sigma[np.ix_(test_S,test_S)] = modi_ld_S

                            test_log_BF=outlier_likelihood(test_S+1, Sigma, z, outlier_tau, len(test_S), 1) - outlier_likelihood(test_S+1, modi_Sigma, z, outlier_tau, len(test_S), 1)
                            test_log_BF = -np.abs(test_log_BF)

                        if np.exp(test_log_BF) < outlier_BF_index:
                            set_gamma[i] = np.delete(set_gamma[i], int(set_star['gamma_set_index'][i]), axis=0)
                            set_gamma_margin[i] = np.delete(set_gamma_margin[i], int(set_star['gamma_set_index'][i]), axis=0)
                            conditional_S = np.concatenate([conditional_S, np.setdiff1d(test_S, working_S)])
                            conditional_S = np.unique(conditional_S).astype(int).tolist()
                        else:
                            break


            if len(working_S) == num_causal:
                set_star = set_star.drop(1)
                aa = set_star['margin']- max(set_star['margin'])
                sec_sample = np.random.choice([0, 2], 1, p=np.exp(aa) / np.sum(np.exp(aa)))
                ind_sec=int(set_star["gamma_set_index"][set_star['set_index']==int(sec_sample)])
                S=set_gamma[sec_sample[0]][ind_sec].tolist()
            else:
                aa = set_star['margin']- max(set_star['margin'])
                sec_sample = np.random.choice(range(0, 3), 1, p=np.exp(aa) / np.sum(np.exp(aa)))
                S=set_gamma[sec_sample[0]][int(set_star['gamma_set_index'][sec_sample[0]])].tolist()

        for item in conditional_S:
            if item not in S:
                S.append(item)
    #END h_ind loop
    #
    if conditional_S is not None:
        all_c_index = []
        l = [s.split(",") for s in B_list["matrix_gamma"]]
        for tt in conditional_S:
            tt=str(tt)
            ind = [i for i, sublist in enumerate(l) if tt in sublist]
            all_c_index.extend(ind)

        all_c_index = list(set(all_c_index))

        if len(all_c_index) > 0:
            temp_B_list = B_list.copy()
            temp_B_list = B_list.drop(all_c_index)
        else:
            temp_B_list = B_list.copy()
    else:
        temp_B_list = B_list.copy()

    result_B_list=temp_B_list=temp_B_list[:min(B, len(temp_B_list))]

    rb1 = result_B_list["set_gamma_margin"]

    difference = abs(rb1[:(len(rb1)//4)].mean()-stored_bf)

    if difference < epsilon:
        break
    else:
        stored_bf = rb1[:(len(rb1)//4)].mean()

    

In [9]:
conditional_S

[5]

In [10]:
S

[0, 3, 5]

In [11]:
working_S

array([0, 4])

In [12]:
h_ind

9

In [13]:
ind_inner_iter

1

In [14]:
epsilon

0.00021

[5]